In [1]:
import subprocess
import logging
import os
import random
import shutil
import platform
from typing import(
    Dict, 
    List, 
    Optional, 
    Tuple,
    Union
)

In [2]:
class File(object):
    '''Creates File object that encapsulates a number of methods and properites
    for file and filename handling.
    
    Attributes (class and instance attributes):
        file (class and instance): Class variable that is set once class is instantiated.
    '''
    
    file: str = ""
    
    def __init__(self,
                 file: str,
                 ext: str = "") -> None:
        '''Init doc-string for File object class.
        
        Usage example:
            >>> with File("file_name.txt") as file:
            ...     file.touch()
            ...     file.write_txt("some text")
            ...
            >>> # or
            >>> 
            >>> file = File("file_name.txt")
            >>> file
            "file_name.txt"

        Args:
            file: Input file (need not exist at runtime/instantiated).
            ext: File extension of input file. If no extension is provided, it
                is inferred.
        '''
        self.file: str = file
        if ext:
            self.ext: str = ext
        elif '.gz' in self.file:
            self.ext: str = self.file[-(7):]
        else:
            self.ext: str = self.file[-(4):]
    
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, traceback):
        return False

    def __repr__(self):
        return self.file
        
    def touch(self) -> None:
        '''Creates empty file.
        
        Usage example:
            >>> file_obj = File("file_name.txt")
            >>> file_obj.touch()   # Creates empty file
        '''
        with open(self.file,'w') as tmp_file:
            pass
        return None
    
    def abs_path(self) -> str:
        '''Returns absolute path of file.
        
        Usage example:
            >>> file_obj = File("file_name.txt")
            >>> file_obj.abs_path()
            "abspath/to/file_namt.txt"
        '''
        if os.path.exists(self.file):
            return os.path.abspath(self.file)
        else:
            self.touch()
            file_path = os.path.abspath(self.file)
            os.remove(self.file)
            return file_path
    
    def rm_ext(self,
              ext: str = "") -> str:
        '''Removes file extension from the file.
        
        Usage example:
            >>> file_obj = File("file_name.txt")
            >>> file_obj.rm_ext() 
            "file_name"
        
        Args:
            ext: File extension.
        
        Returns:
            file: File name with no extension.
        '''
        if ext:
            ext_num: int = len(ext)
            return self.file[:-(ext_num)]
        elif self.ext:
            ext_num = len(self.ext)
            return self.file[:-(ext_num)]
        else:
            return self.file[:-(4)]
        
    def write_txt(self,
                 txt: str = "") -> None:
        '''Writes/appends text to file.
        
        Usage example:
            >>> file_obj = File("file_name.txt")
            >>> file_obj.write_txt("<Text to be written>")
        
        Args:
            txt: Text to be written to file.

        Returns:
            file: File with text written/appended to it.
        '''
        with open(self.file,"a") as tmp_file:
            tmp_file.write(txt)
            tmp_file.close()
        return None

    def file_parts(self,
                  ext: str = "") -> Tuple[str,str,str]:
        '''Splits a file and its path into its constituent 
        parts:
            * file path
            * filename
            * extension
        
        Usage example:
            >>> file_obj = File("file_name.txt")
            >>> file_obj.file_parts()
            ("path/to/file", "filename", ".ext")   # .ext = file extension
        
        Args:
            ext: File extension, needed if the file extension of file 
                object is longer than 4 characters.
        
        Returns:
            path: Absolute file path, excluding filename.
            filename: Filename, excluding extension.
            ext: File extension.
        '''
        
        file: str = self.file
        file = self.abs_path()
        
        if platform.system().lower() == "windows":
            [path, _filename] = os.path.splitdrive(file)
        else:
            [path, _filename] = os.path.split(file)
        
        if ext:
            ext_num = len(ext)
            _filename = _filename[:-(ext_num)]
            [filename, _ext] = os.path.splitext(_filename)
        elif self.ext:
            ext = self.ext
            ext_num = len(ext)
            _filename = _filename[:-(ext_num)]
            [filename, _ext] = os.path.splitext(_filename)
        else:
            [filename, ext] = os.path.splitext(_filename)
        
        return path, filename, ext

In [77]:
class TmpDir(object):
    '''Temporary directory class that creates temporary directories and files
    given a parent directory.
    
    Attributes (class and instance attributes):
        tmp_dir (class and instance): Temproary directory.
        parent_tmp_dir (class and instance): Input parent directory.
    '''
    
    # Set parent tmp directory, as tmp_dir is overwritten
    tmp_dir: str = ""
    parent_tmp_dir: str = ""
    
    def __init__(self,
                tmp_dir: str,
                use_cwd: bool = False) -> None:
        '''Init doc-string for TmpDir class.
        
        Usage example:
            >>> with TmpDir("/path/to/temporary_directory",False) as tmp_dir:
            ...     tmp_dir.mk_tmp_dir()
            ...     # do more stuff
            ...     tmp_dir.rm_tmp_dir(rm_parent=False)
            ...
            >>> # or
            >>> tmp_dir = TmpDir("/path/to/temporary_directory")
            >>> tmp_dir
            "/path/to/temporary_directory"
            >>> tmp_dir.rm_tmp_dir(rm_parent=False)
        
        Args:
            tmp_dir: Temporary parent directory name/path.
            use_cwd: Use current working directory as working direcory.
        '''
        _n: int = 10000 # maximum N for random number generator
        tmp_dir: str = os.path.join(tmp_dir,'tmp_dir_' + 
                               str(random.randint(0,_n)))
        self.tmp_dir: str = tmp_dir
        self.parent_tmp_dir: str = os.path.dirname(self.tmp_dir)
        if use_cwd:
            _cwd = os.getcwd()
            self.tmp_dir = os.path.join(_cwd,self.tmp_dir)
            self.parent_tmp_dir = os.path.dirname(self.tmp_dir)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, traceback):
        return False
    
    def __repr__(self):
        return self.tmp_dir
        
    def mk_tmp_dir(self) -> None:
        '''Creates/makes temporary directory.
        
        Usage example:
            >>> tmp_directory = TmpDir("/path/to/temporary_directory")
            >>> tmp_directory.mk_tmp_dir()
        '''
        if not os.path.exists(self.tmp_dir):
            return os.makedirs(self.tmp_dir)
        else:
            print("Temporary directory already exists")
        
    def rm_tmp_dir(self,
                  rm_parent: bool = False) -> None:
        '''Removes temporary directory.
        
        Usage example:
            >>> tmp_directory = TmpDir("/path/to/temporary_directory")
            >>> tmp_directory.rm_tmp_dir() 
        
        Args:
            rm_parent: Removes parent directory as well.
        '''
        if rm_parent and os.path.exists(self.parent_tmp_dir):
            return shutil.rmtree(self.parent_tmp_dir)
        elif os.path.exists(self.tmp_dir):
            return shutil.rmtree(self.tmp_dir)
        else:
            print("Temporary directory does not exist")
    
    class TmpFile(File):
        '''Child/sub-class of TmpDir. Creates temporary files by inheriting File object
        methods and properties from the File class.
        
        Attributes (class and instance attributes):
            tmp_file (class and instance): Temporary file name.
            tmp_dir (class and instance): Temporary directory name.
        '''
        
        tmp_file: File = ""
        tmp_dir: str = ""

        def __init__(self,
                     tmp_dir: str,
                     tmp_file: Optional[str] = "",
                     ext: Optional[str] = "",
                    ) -> None:
            '''Init doc-string for TmpFile class. Allows for creation of 
            a temporary file in parent class' temporary directory location.
            TmpFile also inherits methods from the File class.
            
            Usage example:
                >>> tmp_directory = TmpDir("/path/to/temporary_directory")
                >>> temp_file = TmpDir.TmpFile(tmp_directory.tmp_dir)
                >>> temp_file
                "/path/to/temporary_directory/temporary_file"
            
            Args:
                tmp_dir: Temporary directory name.
                tmp_file: Temporary file name.
                ext: Temporary directory file extension.
            '''
            self.tmp_dir: str = tmp_dir
            
            if tmp_file:
                self.tmp_file: str = tmp_file
            else:
                _n: int = 10000 # maximum N for random number generator
                self.tmp_file: str = "tmp_file_" + str(random.randint(0,_n))
            
            if ext:
                self.tmp_file: str = self.tmp_file + f".{ext}"

            self.tmp_file = os.path.join(self.tmp_dir,self.tmp_file)
            File.__init__(self,self.tmp_file)
            print(self.tmp_dir)

In [85]:
tmp = TmpDir("tmp_dir",use_cwd=False)

In [86]:
tmp.mk_tmp_dir()

In [87]:
tmp.tmp_dir

'tmp_dir/tmp_dir_3540'

In [88]:
tmp_file = tmp.TmpFile(tmp_dir=tmp.tmp_dir,ext="txt")

tmp_dir/tmp_dir_3540


In [89]:
tmp_file.touch()

In [90]:
tmp_file

tmp_dir/tmp_dir_3540/tmp_file_5307.txt

In [91]:
tmp.rm_tmp_dir(rm_parent=True)